import pandas as pd

In [3]:
fake_news_file_path = '../Intro to ML/fake_news_dataset.csv'
fake_news_data = pd.read_csv(fake_news_file_path)
fake_news_data.head()

,id,title,author,text,state,date_published,source,category,sentiment_score,word_count,...,num_shares,num_comments,political_bias,fact_check_rating,is_satirical,trust_score,source_reputation,clickbait_score,plagiarism_score,label
0,1,Breaking News 1,Jane Smith,This is the content of article 1. It contains ...,Tennessee,30-11-2021,The Onion,Entertainment,-0.22,1302,...,47305,450,Center,FALSE,1,76,6,0.84,53.35,Fake
1,2,Breaking News 2,Emily Davis,This is the content of article 2. It contains ...,Wisconsin,02-09-2021,The Guardian,Technology,0.92,322,...,39804,530,Left,Mixed,1,1,5,0.85,28.28,Fake
2,3,Breaking News 3,John Doe,This is the content of article 3. It contains ...,Missouri,13-04-2021,New York Times,Sports,0.25,228,...,45860,763,Center,Mixed,0,57,1,0.72,0.38,Fake
3,4,Breaking News 4,Alex Johnson,This is the content of article 4. It contains ...,North Carolina,08-03-2020,CNN,Sports,0.94,155,...,34222,945,Center,TRUE,1,18,10,0.92,32.20,Fake
4,5,Breaking News 5,Emily Davis,This is the content of article 5. It contains ...,California,23-03-2022,Daily Mail,Technology,-0.01,962,...,35934,433,Right,Mixed,0,95,6,0.66,77.70,Real


In [5]:
fake_news_data.columns

Index(['id', 'title', 'author', 'text', 'state', 'date_published', 'source',
       'category', 'sentiment_score', 'word_count', 'char_count', 'has_images',
       'has_videos', 'readability_score', 'num_shares', 'num_comments',
       'political_bias', 'fact_check_rating', 'is_satirical', 'trust_score',
       'source_reputation', 'clickbait_score', 'plagiarism_score', 'label'],
      dtype='object')

In [6]:
fake_news_data_na = fake_news_data.dropna(axis=0)

In [7]:
y=fake_news_data_na.label

In [10]:
fake_news_features = ['sentiment_score','readability_score', 'num_shares', 'num_comments','political_bias', 'fact_check_rating', 'is_satirical', 'trust_score',
       'source_reputation', 'clickbait_score', 'plagiarism_score']

X = fake_news_data[fake_news_features]

X.describe()

,sentiment_score,readability_score,num_shares,num_comments,is_satirical,trust_score,source_reputation,clickbait_score,plagiarism_score
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000
mean,-0.000645,54.764595,25144.596750,489.870250,0.497000,49.960750,5.54925,0.494447,50.598110
std,0.574768,14.404027,14387.537467,287.435733,0.500054,29.467911,2.87422,0.289138,28.932298
min,-1.000000,30.020000,39.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.040000
25%,-0.490000,42.480000,12781.750000,238.000000,0.000000,24.000000,3.00000,0.240000,25.915000
50%,-0.010000,54.235000,25308.500000,483.000000,0.000000,50.000000,6.00000,0.490000,51.480000
75%,0.510000,67.215000,37453.500000,741.000000,1.000000,76.000000,8.00000,0.740000,75.580000
max,1.000000,79.980000,50000.000000,1000.000000,1.000000,100.000000,10.00000,1.000000,99.950000


In [19]:
from sklearn.preprocessing import LabelEncoder

# Apply ordinal encoder 
le =LabelEncoder()
labely = le.fit_transform(y)

from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, labely, random_state = 0)


In [23]:
s = (train_X.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)
print(fake_news_data[object_cols].head)

Categorical variables:
['political_bias', 'fact_check_rating']
<bound method NDFrame.head of      political_bias fact_check_rating
0            Center             FALSE
1              Left             Mixed
2            Center             Mixed
3            Center              TRUE
4             Right             Mixed
...             ...               ...
3995          Right             Mixed
3996           Left             FALSE
3997           Left             FALSE
3998           Left             FALSE
3999          Right              TRUE

[4000 rows x 2 columns]>


In [25]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_X[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(val_X[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = train_X.index
OH_cols_valid.index = val_X.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_X.drop(object_cols, axis=1)
num_X_valid = val_X.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

In [27]:
from sklearn.tree import DecisionTreeRegressor

fake_news_model = DecisionTreeRegressor(random_state = 0)
fake_news_model.fit(OH_X_train, train_y)
prediction = fake_news_model.predict(OH_X_valid)

print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(prediction)

Making predictions for the following 5 houses:
   sentiment_score  readability_score  num_shares  num_comments  \
0            -0.22              66.18       47305           450   
1             0.92              41.10       39804           530   
2             0.25              30.04       45860           763   
3             0.94              75.16       34222           945   
4            -0.01              43.90       35934           433   

  political_bias fact_check_rating  is_satirical  trust_score  \
0         Center             FALSE             1           76   
1           Left             Mixed             1            1   
2         Center             Mixed             0           57   
3         Center              TRUE             1           18   
4          Right             Mixed             0           95   

   source_reputation  clickbait_score  plagiarism_score  
0                  6             0.84             53.35  
1                  5             0.85      

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(val_y, prediction)